In [7]:
from pyspark import SparkContext
import sys
import csv
from time import time
from collections import Counter
from itertools import combinations

In [8]:
input_file = "/Users/liangsiqi/Documents/Dataset/ta_feng_dataset/ta_feng_all_months_merged.csv"

In [9]:
def extract_date_customer_product(iterator):
    pairs = []
    for line in iterator:
        pairs.append([line[0]+'-'+line[1], int(line[5])])
    return pairs

def cnt(iterator):
    yield len(list(iterator))
    
def save_pairs(pair,file):
    print(pair[0] + ',' + '%d' % pair[1],file=file)
    
    
    
def sort_into_file(reduce_result, tag, file=None):
    inter_result = [i.split('_') for i in reduce_result]
    max_size = max([len(i) for i in inter_result])
    candidate_itemsets = {i: [] for i in range(1, max_size + 1)}
    [candidate_itemsets[len(i)].append(i) for i in inter_result]

    for i in range(1, max_size + 1):
        itemsets_size_i = [tuple(x) for x in candidate_itemsets[i]]
        candidate_itemsets[i] = sorted(itemsets_size_i, key=lambda x: [i for i in x])  # sorted in lexicographical order

    print(tag, file=file)
    for i in range(1, max_size + 1):
        itemsets = candidate_itemsets[i]
        if i == 1:
            set_cnt = len(itemsets)
            for j in range(set_cnt):
                if j < set_cnt - 1:
                    print("('%s')" % itemsets[j], end=",", file=file)
                else:
                    print("('%s')" % itemsets[j], file=file)
        else:
            set_cnt = len(itemsets)
            for j in range(set_cnt):
                if j < set_cnt - 1:
                    print(itemsets[j], end=",", file=file)
                else:
                    print(itemsets[j], file=file)
        if i != max_size:
            print("", file=file)


def candidate_count(baskets, candidate_itemsets):
    itemsets_count = []
    baskets = list(baskets)
    for basket in baskets:
        for itemset in candidate_itemsets:
            if all([item in basket for item in itemset.split('_')]):
                itemsets_count.append(itemset)
    return Counter(itemsets_count).items()


def A_priori_short_basket(baskets, support, total_baskets_cnt):
    clean_baskets = list(baskets)
    local_support = support * len(clean_baskets) / float(total_baskets_cnt)
    baskets_cnt_ordered = sorted(Counter([len(basket) for basket in clean_baskets]).items(), reverse=True)
    # Reduce max_itemset_size, if the number of baskets with certain length is less than local_support,
    # then the maximum size of frequent itemset must be smaller than the length of such baskets
    acc_sum = 0
    for cnt_item in baskets_cnt_ordered:
        max_itemset_size = cnt_item[0]
        acc_sum += cnt_item[1]
        if acc_sum >= local_support:
            break
        else:
            continue
    candidate_itemsets = []

    # frequent items
    single_cnt = Counter()
    for basket in clean_baskets:
        single_cnt.update(basket)

    candidate_single = sorted([i for i in single_cnt if single_cnt[i] >= local_support])
    former_candidate = candidate_single
    candidate_itemsets.extend(former_candidate)

    # frequent itemsets with size >= 2
    for itemset_size in range(2, max_itemset_size + 1):
        itemset_cnt = {}
        for basket in clean_baskets:
            if len(basket) < itemset_size:
                continue
            for itemset in combinations(basket, itemset_size):
                itemset = sorted(itemset)
                if itemset_size == 2:
                    immediate_subset_flag = True
                    for immediate_subset in combinations(itemset, itemset_size - 1):
                        if immediate_subset[0] not in former_candidate:
                            immediate_subset_flag = False
                            break
                else:
                    immediate_subset_flag = True
                    for immediate_subset in combinations(itemset, itemset_size - 1):
                        if '_'.join(sorted(immediate_subset)) not in former_candidate:
                            immediate_subset_flag = False
                            break
                if immediate_subset_flag:
                    itemset_string = '_'.join(itemset)
                    if itemset_string not in itemset_cnt:
                        itemset_cnt[itemset_string] = 0
                    itemset_cnt[itemset_string] += 1
        # keep itemset in form of id1_id2_id3 for sake of later hash
        former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= local_support]
        candidate_itemsets.extend(former_candidate)

    return candidate_itemsets


def A_priori_long_basket(baskets, support, total_baskets_cnt):
    clean_baskets = list(baskets)
    local_support = support * len(clean_baskets) / float(total_baskets_cnt)
    baskets_cnt_ordered = sorted(Counter([len(basket) for basket in clean_baskets]).items(), reverse=True)
    # Reduce max_itemset_size, if the number of baskets with certain length is less than local_support,
    # then the maximum size of frequent itemset must be smaller than the length of such baskets
    acc_sum = 0
    for cnt_item in baskets_cnt_ordered:
        max_itemset_size = cnt_item[0]
        acc_sum += cnt_item[1]
        if acc_sum >= local_support:
            break
        else:
            continue
    candidate_itemsets = []

    # frequent items
    single_cnt = Counter()
    for basket in clean_baskets:
        single_cnt.update(basket)

    candidate_single = sorted([i for i in single_cnt if single_cnt[i] >= local_support])
    former_candidate = candidate_single
    candidate_itemsets.extend(former_candidate)

    # frequent pairs
    itemset_cnt = Counter()
    former_possible_items_set = former_candidate
    for pair in combinations(former_possible_items_set, 2):
        pair = sorted(pair)
        for basket in clean_baskets:
            contain_pair_flag = True
            if len(basket) < 2:
                continue
            for item in pair:
                if item not in basket:
                    contain_pair_flag = False
                    break
            if contain_pair_flag:
                itemset_cnt.update(['_'.join(pair)])
    former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= local_support]
    candidate_itemsets.extend(former_candidate)
    if len(former_candidate) < 2:
        return candidate_itemsets

    # frequent itemsets with size >= 3
    for itemset_size in range(3, max_itemset_size + 1):
        itemset_cnt = Counter()
        possible_item_set = []
        [possible_item_set.extend(i.split('_')) for i in former_candidate]
        possible_item_set = list(set(possible_item_set))
        itemsets_checked = []
        for smaller_set_string in former_candidate:
            smaller_set = smaller_set_string.split('_')
            possible_to_add = set(possible_item_set) - set(smaller_set)
            for item_to_add in possible_to_add:
                new_itemset = sorted(smaller_set + [item_to_add])
                new_itemset_string = '_'.join(new_itemset)
                if new_itemset_string not in itemsets_checked:
                    itemsets_checked.append(new_itemset_string)
                    immediate_subset = ['_'.join(sorted(i)) for i in combinations(new_itemset, itemset_size - 1)]
                    all_subset_in_former = all([i in former_candidate for i in immediate_subset])
                    if all_subset_in_former:
                        for basket in clean_baskets:
                            if all([i in basket for i in new_itemset]):
                                itemset_cnt.update([new_itemset_string])
        former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= local_support]
        candidate_itemsets.extend(former_candidate)
        if len(former_candidate) < 2:
            break

    return candidate_itemsets


In [11]:
sc = SparkContext.getOrCreate()
raw_data = sc.textFile(input_file)
header = raw_data.first()
raw_data_without_header = raw_data.filter(lambda x: x != header)
# "TRANSACTION_DT","CUSTOMER_ID","AGE_GROUP","PIN_CODE","PRODUCT_SUBCLASS","PRODUCT_ID","AMOUNT","ASSET","SALES_PRICE"
clean_raw_data = raw_data_without_header.mapPartitions(lambda x: csv.reader(x))
preprocessed_data = clean_raw_data.mapPartitions(lambda x: extract_date_customer_product(x))
preprocess_result = preprocessed_data.collect()
with open("./customer_product.csv",'w') as out_f:
    print("DATE-CUSTOMER_ID,PRODUCT_ID",file=out_f)
    for pair in preprocess_result:
        print(pair[0] + ',' + '%d' % pair[1],file=out_f)
sc.stop()

In [4]:
filter_threshold = 20
case = 1
support = 50

def task2(case,output_file,minPartition=2,support=50,filter_threshold=20):
    sc = SparkContext.getOrCreate()
    start = time()
    raw_data = sc.textFile("./customer_product.csv",minPartition)
    header = raw_data.first()
    raw_data_without_header = raw_data.filter(lambda x: x != header)
    clean_data = raw_data_without_header.mapPartitions(lambda x: csv.reader(x))
    baskets = clean_data.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda a, b: a + b).filter(lambda x: len(x[1]) >= filter_threshold)
    clean_baskets = baskets.map(lambda x: sorted(list(set(list(x[1])))))
    clean_baskets.persist()
    total_baskets_cnt = clean_baskets.count()
#     print("Baskets before filter: %d\nBaskets after filter: %d" % (baskets.count(),clean_baskets.count()))

    # Phase 1: get candidate itemsets from all subsets (different partitions)
    if case == 1:
        phase1_map = clean_baskets.mapPartitions(
            lambda baskets_subset: A_priori_short_basket(baskets_subset, support, total_baskets_cnt)).map(
            lambda x: (x, 1))
    else:
        phase1_map = clean_baskets.mapPartitions(
            lambda baskets_subset: A_priori_long_basket(baskets_subset, support, total_baskets_cnt)).map(
            lambda x: (x, 1))
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: 1).keys().collect()  # remove duplicates, Intermediate result

    # phase 2: count candidate itemsets and filter out using support
    phase2_map = clean_baskets.mapPartitions(lambda baskets_subset: candidate_count(baskets_subset, phase1_reduce))
    phase2_reduce = phase2_map.reduceByKey(lambda x, y: x + y).filter(lambda kv: kv[1] >= support).keys().collect()

    with open(output_file, 'w') as f:
        sort_into_file(phase1_reduce, "Candidates:", f)
        print("", file=f)
        sort_into_file(phase2_reduce, "Frequent Itemsets:", f)

    end = time()
    sc.stop()
    
    
    print("Duration: %d" % int(end - start))

In [21]:
%timeit task2(2,"./local_task2.csv",minPartition=2)
# Duration: 343
# Duration: 344
# Duration: 350

In [16]:
%timeit task2(2,"./local_task2_p_3.csv",minPartition=3)

Baskets before filter: 119578
Baskets after filter: 6477
Duration: 276


In [17]:
%timeit task2(2,"./local_task2_p_4.csv",minPartition=4)

Baskets before filter: 119578
Baskets after filter: 6477
Duration: 226


In [27]:
%timeit task2(2,"./local_task2_p_5.csv",minPartition=5)
# Duration: 216
# Duration: 227
# Duration: 223
# Duration: 218
# Duration: 218
# Duration: 216

In [29]:
%timeit task2(2,"./local_task2_p_6.csv",minPartition=6)
# Duration: 220
# Duration: 212
# Duration: 206
# Duration: 204
# Duration: 202
# Duration: 208

In [6]:
%timeit task2(2,"./local_task2_p_10.csv",minPartition=10)
# Duration: 234

In [7]:
def test1(minPartition=2):
    sc = SparkContext.getOrCreate()
    raw_data = sc.textFile(input_file,minPartition)
    header = raw_data.first()
    raw_data_without_header = raw_data.filter(lambda x: x != header)
    # "TRANSACTION_DT","CUSTOMER_ID","AGE_GROUP","PIN_CODE","PRODUCT_SUBCLASS","PRODUCT_ID","AMOUNT","ASSET","SALES_PRICE"
    clean_raw_data = raw_data_without_header.mapPartitions(lambda x: csv.reader(x))
    preprocessed_data = clean_raw_data.mapPartitions(lambda x: extract_date_customer_product(x))
    res = preprocessed_data.collect()
    sc.stop()
    
def test2(minPartition=2):
    sc = SparkContext.getOrCreate()
    raw_data = sc.textFile(input_file,minPartition)
    header = raw_data.first()
    raw_data_without_header = raw_data.filter(lambda x: x != header)
    # "TRANSACTION_DT","CUSTOMER_ID","AGE_GROUP","PIN_CODE","PRODUCT_SUBCLASS","PRODUCT_ID","AMOUNT","ASSET","SALES_PRICE"
    clean_raw_data = raw_data_without_header.mapPartitions(lambda x: csv.reader(x))
    preprocessed_data = clean_raw_data.map(lambda record: [record[0]+'-'+record[1], int(record[5])])
    res = preprocessed_data.collect()
    sc.stop()

In [8]:
print("mapPartitions:")
print("minPartition=2:  ",end='')
%timeit test1(2)
print("minPartition=3:  ",end='')
%timeit test1(3)
print("minPartition=4:  ",end='')
%timeit test1(4)
print("minPartition=5:  ",end='')
%timeit test1(5)
print("minPartition=6:  ",end='')
%timeit test1(6)
print("minPartition=7:  ",end='')
%timeit test1(7)
print("minPartition=8:  ",end='')
%timeit test1(8)
print("minPartition=9:  ",end='')
%timeit test1(9)
print("minPartition=10: ",end='')
%timeit test1(10)
print("minPartition=11: ",end='')
%timeit test1(11)
print("minPartition=12: ",end='')
%timeit test1(12)
print("minPartition=14: ",end='')
%timeit test1(14)
print("minPartition=15: ",end='')
%timeit test1(15)
print("minPartition=16: ",end='')
%timeit test1(16)
print("minPartition=17: ",end='')
%timeit test1(17)
print("minPartition=18: ",end='')
%timeit test1(18)
print("minPartition=19: ",end='')
%timeit test1(19)
print("minPartition=20: ",end='')
%timeit test1(20)
print("minPartition=30: ",end='')
%timeit test1(30)
print("minPartition=40: ",end='')
%timeit test1(40)
print("minPartition=50: ",end='')
%timeit test1(50)
print("minPartition=60: ",end='')
%timeit test1(60)

mapPartitions:
minPartition=2:  2.79 s ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=3:  2.36 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=4:  2.17 s ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=5:  2.08 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=6:  1.97 s ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=7:  1.99 s ± 35.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=8:  1.99 s ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=9:  1.96 s ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=10: 1.97 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=11: 1.96 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=12: 1.95 s ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=14: 2.05 s ± 18 ms per loop (mean ± std

In [9]:
print("minPartition=70: ",end='')
%timeit test1(70)
print("minPartition=80: ",end='')
%timeit test1(80)
print("minPartition=90: ",end='')
%timeit test1(90)
print("minPartition=The 100: ",end='')
%timeit test1(100)
print("minPartition=120: ",end='')
%timeit test1(120)
print("minPartition=140: ",end='')
%timeit test1(140)
print("minPartition=160: ",end='')
%timeit test1(160)

minPartition=70: 1.89 s ± 7.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=80: 1.91 s ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=90: 1.93 s ± 28.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=The 100: 1.95 s ± 31.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=120: 1.98 s ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=140: 1.98 s ± 44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=160: 2.02 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
print("map:")
print("minPartition=2:  ",end='')
%timeit test2(2)
print("minPartition=3:  ",end='')
%timeit test2(3)
print("minPartition=4:  ",end='')
%timeit test2(4)
print("minPartition=5:  ",end='')
%timeit test2(5)
print("minPartition=6:  ",end='')
%timeit test2(6)
print("minPartition=7:  ",end='')
%timeit test2(7)
print("minPartition=8:  ",end='')
%timeit test2(8)
print("minPartition=9:  ",end='')
%timeit test2(9)
print("minPartition=10: ",end='')
%timeit test2(10)
print("minPartition=11: ",end='')
%timeit test2(11)
print("minPartition=12: ",end='')
%timeit test2(12)
print("minPartition=14: ",end='')
%timeit test2(14)
print("minPartition=15: ",end='')
%timeit test2(15)
print("minPartition=16: ",end='')
%timeit test2(16)
print("minPartition=17: ",end='')
%timeit test2(17)
print("minPartition=18: ",end='')
%timeit test2(18)
print("minPartition=19: ",end='')
%timeit test2(19)
print("minPartition=20: ",end='')
%timeit test2(20)
print("minPartition=30: ",end='')
%timeit test2(30)
print("minPartition=40: ",end='')
%timeit test2(40)
print("minPartition=50: ",end='')
%timeit test2(50)
print("minPartition=60: ",end='')
%timeit test2(60)
print("minPartition=70: ",end='')
%timeit test2(70)
print("minPartition=80: ",end='')
%timeit test2(80)
print("minPartition=90: ",end='')
%timeit test2(90)
print("minPartition=100: ",end='')
%timeit test2(100)
print("minPartition=120: ",end='')
%timeit test2(120)
print("minPartition=140: ",end='')
%timeit test2(140)
print("minPartition=160: ",end='')
%timeit test2(160)

map:
minPartition=2:  2.66 s ± 38.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=3:  2.27 s ± 29.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=4:  2.1 s ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=5:  1.97 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=6:  1.96 s ± 31.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=7:  1.95 s ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=8:  1.94 s ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=9:  1.94 s ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=10: 1.94 s ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=11: 1.95 s ± 39.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=12: 1.94 s ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
minPartition=14: 2.04 s ± 25.7 ms per loop (mean ± std. dev. of

In [12]:
sc = SparkContext.getOrCreate()
raw_data = sc.textFile("./customer_product.csv",5)
header = raw_data.first()
raw_data_without_header = raw_data.filter(lambda x: x != header)
clean_data = raw_data_without_header.mapPartitions(lambda x: csv.reader(x))
baskets = clean_data.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda a, b: a + b).filter(lambda x: len(x[1]) >= filter_threshold)
clean_baskets = baskets.map(lambda x: sorted(list(set(list(x[1])))))
clean_baskets.persist()
frequent_pairs = [('37000440147', '37000445111'),('4710011401128', '4710011401135'),('4710011401128', '4710011401142'),('4710011401128', '4710011405133'),('4710011401128', '4710011406123'),('4710011401128', '4710011409056'),('4710011401128', '4710857000059'),('4710011401128', '4714981010038'),('4710011401135', '4710011401142'),('4710011401135', '4710011405133'),('4710011401135', '4710011406123'),('4710011401135', '4710011409056'),('4710011401142', '4710011405133'),('4710011401142', '4710011406123'),('4710011401142', '4710011409056'),('4710011402019', '4710011402026'),('4710011405133', '4710011406123'),('4710011405133', '4710011409056'),('4710011406123', '4710011409056'),('4710018004605', '4710018004704'),('4710085120093', '4710085120628'),('4710085120093', '4710085172696'),('4710085120093', '4710085172702'),('4710085120628', '4710085172696'),('4710085120628', '4710085172702'),('4710085120680', '4710085120697'),('4710085120703', '4710085120710'),('4710085172696', '4710085172702'),('4710088410139', '4710088410610'),('4710088414311', '4710088414328'),('4710114105046', '4710114606048'),('4710114128038', '4710114606048'),('4710114128038', '4711271000014'),('4710254049323', '4710254049521'),('4710466103073', '4710466103080'),('4710908131589', '4711271000014'),('4711271000014', '4714981010038'),('4711524000891', '4711524000907'),('4711524000891', '4711524001041'),('4711524000907', '4711524001041'),('4712019100591', '4712019100607'),('719859796117', '719859796124'),('723125485032', '7231254880206')]
frequent_triple = [('4710011401128', '4710011401135', '4710011401142'),('4710011401128', '4710011401135', '4710011405133'),('4710011401128', '4710011401135', '4710011406123'),('4710011401128', '4710011401135', '4710011409056'),('4710011401128', '4710011401142', '4710011405133'),('4710011401128', '4710011405133', '4710011406123'),('4710011401128', '4710011405133', '4710011409056'),('4710011401128', '4710011406123', '4710011409056'),('4710011401135', '4710011405133', '4710011406123'),('4710011401135', '4710011405133', '4710011409056'),('4710011401135', '4710011406123', '4710011409056'),('4710011405133', '4710011406123', '4710011409056'),('4710085120093', '4710085120628', '4710085172696'),('4710085120093', '4710085120628', '4710085172702'),('4710085120628', '4710085172696', '4710085172702')]
frequent_four = [('4710011401128', '4710011401135', '4710011405133', '4710011406123'),('4710011401128', '4710011401135', '4710011405133', '4710011409056'),('4710011401128', '4710011401135', '4710011406123', '4710011409056'),('4710011401128', '4710011405133', '4710011406123', '4710011409056')]
frequent_itemsets = []
for pair in frequent_pairs:
    frequent_itemsets.append('_'.join(pair))
for triple in frequent_triple:
    frequent_itemsets.append('_'.join(triple))
for four in frequent_four:
    frequent_itemsets.append('_'.join(four))


phase2_map = clean_baskets.mapPartitions(lambda baskets_subset: candidate_count(baskets_subset, frequent_itemsets))
phase2_reduce = phase2_map.reduceByKey(lambda x, y: x + y)
sorted_frequents = phase2_reduce.sortBy(lambda x: x[1], ascending=False).collect()

In [13]:
sorted_frequents
4710011401128: 4710011401135, 4710011405133, 4710011409056, 4710011406123
               (4710011401135,4710011405133),  (4710011401135,4710011409056), (4710011405133,4710011409056)

[('4710011401128_4710011401135', 185),
 ('4710011401128_4710011405133', 182),
 ('4710011401128_4710011409056', 166),
 ('4710011401128_4710011406123', 149),
 ('4710011401135_4710011405133', 139),
 ('4710085120093_4710085120628', 127),
 ('4710011405133_4710011409056', 126),
 ('4710011401135_4710011409056', 124),
 ('4710085120628_4710085172696', 119),
 ('4710011401128_4710011401135_4710011405133', 116),
 ('4710018004605_4710018004704', 116),
 ('4710011405133_4710011406123', 113),
 ('4710085120093_4710085172696', 108),
 ('4710011401128_4710011401135_4710011409056', 106),
 ('4710011401128_4710011405133_4710011409056', 103),
 ('4710011401135_4710011406123', 103),
 ('4710011406123_4710011409056', 96),
 ('4711271000014_4714981010038', 94),
 ('4710011401128_4710011405133_4710011406123', 85),
 ('4710011401135_4710011405133_4710011409056', 84),
 ('4710011401128_4710011401135_4710011406123', 84),
 ('4710085120680_4710085120697', 82),
 ('4710011401128_4710011401142', 81),
 ('4710085120703_471008512